In [10]:
import pandas as pd
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
#mortality_20_topics = pd.read_csv('topic_model_dia_20_topics.csv')
mortality_50_topics = pd.read_csv('topic_model_dia_50_topics.csv')

In [28]:
X = mortality_50_topics.drop(columns=['top_icd'])
y = mortality_50_topics['top_icd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
y_train = y_train.astype('str')
y_test = y_test.astype('str')

In [29]:
kmeans = KMeans(n_clusters = 4).fit(X_train)
kmeans_pred = kmeans.predict(X_test)

In [30]:
ICD9_CODE_map = {
    '414':  0, #chronic heart
    '38':  1, #sepsis
    '410': 2, #heart attack
    '424': 3, #diseases of endocardium
}

#d["ICD9_CODE"] = d["ICD9_CODE"].map(ICD9_CODE_map)
y_train_km = y_train.map(ICD9_CODE_map)
y_test_km = y_test.map(ICD9_CODE_map)
#X = d.values
#X = X[:,1:9]
kmeans = KMeans(n_clusters=4, random_state=0).fit(X_test)

pred = kmeans.labels_
true = y_test_km
#true = d["ICD9_CODE"].values
#pd.DataFrame(kmeans.labels_).to_csv("kmeanlabels.csv")

#4 classes
#pred - kmeans.labels_ - 0/1/2/3
#true - ICD9 

acc=0
max = 0
predTemp=[-1,-1,-1,-1]
predNew=[-1]*len(pred)
predAssign = pd.Series(pred)
predFinal=[-1]*len(pred)

from sklearn.metrics import accuracy_score

for i in range(4):
    predTemp[i]=0
    for j in range(4):
        if j!=i:
            predTemp[j]=1
        else:
            continue
        for k in range(4):
            if k!=i and k!=j:
                predTemp[k]=2
            else:
                continue
            for l in range(4):
                if l!=i and l!=k and l!=j:
                    predTemp[l]=3
                    pred_map = {
                        0: predTemp[0],
                        1: predTemp[1],
                        2: predTemp[2],
                        3: predTemp[3],
                    }
                    predNew = predAssign.map(pred_map)
                    predNew = predNew.values
                    acc = accuracy_score(true, predNew)
                    if acc > max: 
                        max = acc
                        predFinal = predNew  
                else:
                    continue

In [31]:
#Accuracy!
from sklearn.metrics import accuracy_score
print("acc_sklearn:",accuracy_score(true, predFinal))

#Assign new class to pred.
precision_c0,recall_c0,f1_c0=F1(predFinal,true.tolist(),0)
precision_c1,recall_c1,f1_c1=F1(predFinal,true.tolist(),1)
precision_c2,recall_c2,f1_c2=F1(predFinal,true.tolist(),2)
precision_c3,recall_c3,f1_c3=F1(predFinal,true.tolist(),3)

preAvg=(precision_c0+precision_c1+precision_c2+precision_c3)/4
reAvg=(recall_c0+recall_c1+recall_c2+recall_c3)/4
f1Avg=(f1_c0+f1_c1+f1_c2+f1_c3)/4

print(preAvg,reAvg,f1Avg)

acc_sklearn: 0.5398613518197574
0.46040415283311503 0.48061405089397613 0.4614834935291472
